In [1]:
from sklearn.metrics.pairwise import cosine_similarity
from time import sleep
import glob,os
import pandas as pd

In [2]:
from openai import OpenAI
api_key = "***"
organization = '***'
client = OpenAI(api_key=api_key,organization=organization,base_url='https://api.openai.com/v1/')

In [ ]:
assistant = client.beta.assistants.create(
  name="cage table organizer",
  instructions='Follow the instructions, extract the information related to organic molecular cages from the given text and organize them into a table',
  model="gpt-4o"
)

In [22]:
all_manual_table = r"E:\liuming\cage-GPT\all_manual_table.txt"
with open(all_manual_table,'r',encoding='gb18030') as f:
    class1 = f.read()
title_dict = {}
for content in class1.split('\n\n'):
    if '\n' in content:
        title = content.split('\n')[0]
        result = content.split('\n')[1:]
        new_result = ''
        for item in result:
            new_result += item
            new_result += '\n'
        if title != '':
            title = title.replace(':','')
            title_dict[title] = new_result

In [8]:
with open('./level-1-instruction.txt','r',encoding='gb18030') as f:
    description = f.read()

In [6]:
with open('./level-2and3-instruction.txt','r',encoding='gb18030') as f:
    description = f.read()

In [7]:
import numpy as np
np.float = float
np.int = int
np.object = object
np.bool = bool

In [26]:
tables = []
table_name = "name of the organic molecular cage|topology|ccdc number|specific surface area|cage's synthesis procedure|reactant1|reactant1's synthesis procedure|reactant2|reactant2's synthesis procedure|"

In [25]:
csv_files = glob.glob(csv_folder)
for input_csv in csv_files:
    key = input_csv.split('\\')[-1].replace('.csv','')
    if key not in title_dict.keys():
        print(key)

In [27]:
csv_folder = r'E:\liuming\cage-GPT\CSV_file\level3\*.csv'
csv_files = glob.glob(csv_folder)
for input_csv in csv_files:
    thread = client.beta.threads.create()
    key = input_csv.split('\\')[-1].replace('.csv','')
    with open(input_csv,'r',encoding='gb18030',errors='ignore') as f:
        f_content = f.read()
    try:
        message = client.beta.threads.messages.create(thread_id=thread.id, role='user',content=description+f_content)
        GPT_people_answer = title_dict[key]
        run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id
        )
        sleep(30)
        print(run.id)
        run_retrive = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id)
        print(run_retrive.status)
        while(run_retrive.status == 'in_progress'):
            sleep(10)
            run_retrive = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
        print(run_retrive.status)
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        output_message = messages.data[0].content[0].text.value
        i = output_message.find("|")
        j = output_message.rfind("|")
        gpt_answer = output_message[i:j+1].replace('--','').replace('|-|','')
        sentence = 'standard answer:'+table_name+'\n\n'+GPT_people_answer+'GPT answer:'+gpt_answer
        GPT_people_answer = table_name+'\n\n'+GPT_people_answer
        tables.append([input_csv,GPT_people_answer,gpt_answer])
        context_emb1 = client.embeddings.create(model="text-embedding-ada-002",input=GPT_people_answer)
        context_emb2 = client.embeddings.create(model="text-embedding-ada-002",input=gpt_answer)
        context_emb_data1 = context_emb1.data[0].embedding
        context_emb_data2 = context_emb2.data[0].embedding
        cos_similarity = cosine_similarity([context_emb_data1], [context_emb_data2])
        print(input_csv,'finished','cos_similarity=',cos_similarity)
    except Exception as e:
        print(str(e))
        print(key)

run_orA2jgoSfVJf9NGRYDJOp2g9
in_progress
completed
E:\liuming\cage-GPT\CSV_file\level3\2009----Water-soluble octahedral polyammonium nanocapsules synthesisand encapsulation studies.csv finished cos_similarity= [[0.94545304]]
run_4uFVpKBc9eRIKeqkwb9RY5fv
completed
completed
E:\liuming\cage-GPT\CSV_file\level3\2011---Angew. Chem. Int. Ed.--- -CC7-CC8.csv finished cos_similarity= [[0.97295389]]
run_RMcBzP8RpWGPJ8nI0AuOCqra
in_progress
completed
E:\liuming\cage-GPT\CSV_file\level3\2011---Multicomponent Assembly of Cavitand-Based PolyacylhydrazoneNanocapsules.csv finished cos_similarity= [[0.95774487]]
run_3YbUxfrXE0cHiVL3s7OzLGPL
in_progress
completed
E:\liuming\cage-GPT\CSV_file\level3\2011-naseer-et-al-synthesis-and-functionalization-of-heteroatom-bridged-bicyclocalixaromatics-large-molecular.csv finished cos_similarity= [[0.95341601]]
run_DexMXqv4RWwGBA1EKOHgA5NA
completed
completed
E:\liuming\cage-GPT\CSV_file\level3\2013----Chem Comm---Direct gravimetric sensing.csv finished cos_simil

In [30]:
import pandas as pd
df_gpt_manual = pd.DataFrame(tables)
GPT4o_answer = r'E:\liuming\cage-GPT\CSV_file\GPT4o_answer_level3.xlsx'
df_gpt_manual.to_excel(GPT4o_answer)